In [5]:
import smtplib
import time
import calendar
import geopy
from  geopy.geocoders import Nominatim
import dateutil.parser
from datetime import datetime
import pandas as pd
from geopy.distance import great_circle 
import openrouteservice
from geopy.exc import GeocoderTimedOut
from time import gmtime
from time import strftime
df = pd.read_csv("datacopy.csv")

global city

def appointment():
    k = 0
    df = pd.read_csv("datacopy.csv")

    # TAKING THE CUSTOMER QUERY....
    global user_name
    user_name = input("customer Name?")
    user_id = input("customer ID ?")
    profession = input("which  profession service provider you want ?")
    service_provider_name = input("service provider name ?")
    Enter_date=input("Enter starting Date(dd/mm/yyyy):")
    slot_start1="10:00:00"
    slot_end1="12:00:00"
    slot_start2="14:00:00"
    slot_end2="16:00:00"
    slot_start3="18:00:00"
    slot_end3="20:00:00"
    print("Hello sir , Our service Providers have their specific Timing, please Choose from there")
    print("option-1: {} - {}\n".format(slot_start1,slot_end1))
    print("option-2: {} - {}\n".format(slot_start2,slot_end2))
    print("option-3: {} - {}\n".format(slot_start3,slot_end3))
    slot=input("Enter Your Sloting Option:")
    
    if slot=='1':
        
        
        bookingStartTimeperiod =Enter_date+" "+slot_start1
        bookingEndTimeperiod =  Enter_date+" "+slot_end1
        
       
    elif slot=='2':
        bookingStartTimeperiod =Enter_date+" "+slot_start2
        bookingEndTimeperiod =  Enter_date+" "+slot_end2
       
        
        
    elif slot=='3':
        bookingStartTimeperiod =Enter_date+" "+slot_start3
        bookingEndTimeperiod =  Enter_date+" "+slot_end3
        
    else:
        print("choose a valid slot For further Booking..")
        
    global city
    city=input("Customer Location(From) :" )
    global service_city
    
    service_city=input("Service Provider Location(To) :")
    
    day = dateutil.parser.parse(bookingStartTimeperiod).date()  
    if  day.weekday()<5:
        
        
        for j in range(len(df["service_provider_name"])):
            
            

            if df["service_provider_name"][j]==service_provider_name and profession==df["profession"][j] and ((df["bookingStartTimeperiod"][j]<bookingStartTimeperiod and df["bookingEndTimeperiod"][j]>=bookingStartTimeperiod)or(df["bookingStartTimeperiod"][j]<bookingEndTimeperiod and df["bookingEndTimeperiod"][j]>=bookingEndTimeperiod)):
                

                k=1
                break        
        if(k==0):
            Availablestatus="Booked"

            print("Available..your solt  booking is in progresss....")

            b=pd.DataFrame([[user_name,user_id,profession,service_provider_name,(df["service_provider_id"][len(df)-1])+1,bookingStartTimeperiod,bookingEndTimeperiod,city,Availablestatus]])
            b.columns=df.columns
            df=df.append(b)
        # ADDING THE AVAILABLE QUERY IN THE DATA RECORD

            df.to_csv("datacopy.csv",index=False)
        
            time.sleep(2)
            msg = "Congrats,"+user_name+"\n your slot is booked."
            print(msg)

            Calculate_distance()
            print("Thank You so much, See you again")
            cancle()
        else:
            status="Not_Available"
            msg1="Sorry!! "+user_name+" service provider is not avaiable\n "
            print(msg1)
            if slot=='1':
                print("Here is some Suggested  Slot you Can choose")
                print("option-2: {} - {}\n".format(slot_start2,slot_end2))
                print("option-3: {} - {}\n".format(slot_start3,slot_end3))
            elif slot=='2':
                
                print("Here is some Suggested  Slot you Can choose")
                print("option-1: {} - {}\n".format(slot_start1,slot_end1))
                print("option-3: {} - {}\n".format(slot_start3,slot_end3))
            elif slot=='3':
                
                
                print("Here is some Suggested  Slot you Can choose")
                print("option-1: {} - {}\n".format(slot_start1,slot_end1))
                print("option-2: {} - {}\n".format(slot_start2,slot_end2))
        # ADDING THE NOT_AVAILABLE QUERY IN THE DATA RECORD
            b=pd.DataFrame([[user_name,user_id,profession,service_provider_name,(df["service_provider_id"][len(df)-1])+1,bookingStartTimeperiod,bookingEndTimeperiod,city,status]])
            b.columns=df.columns
            df=df.append(b)
            df.to_csv("datacopy.csv",index=False)
            cancle()
    else:
        print("The date you have entered is weekend. Our service provider is not available at that date")       

def cancle():
    
    cancle_booking=input("Do u want to cancel Your any existing Booking? (y/n)")
    if cancle_booking=='y':
        
                
        user=input("Enter Booking name:")
        df = pd.read_csv("datacopy.csv")
        
        df=df.set_index("user_name")
        df=df.drop([user],axis=0)
        df.to_csv('datacopy.csv')
        print("successFully cancled That Appointment.")
        
        
    else:
        print(".................................................................")
        
def Calculate_distance():
    
    
    my_address = city
    my_address2=service_city

    geolocator = Nominatim(timeout=None)
    
    try:
        
        location = geolocator.geocode(my_address,timeout=10)
        location2 = geolocator.geocode(my_address2,timeout=10)
    
        lat1=location.latitude
        long1=location.longitude
        lat2=location2.latitude
        long2=location2.longitude
  
        startdis=(lat1,long1)
        enddis=(lat2,long2)
        start=(long1,lat1)
        end=(long2,lat2)
        coords=(start,end)
        client = openrouteservice.Client(key='5b3ce3597851110001cf6248b5e8d9f3682f46b4adc873c43edea893') 
        routes = client.directions(coords)
        distance=great_circle(startdis, enddis).km
    
        r=routes['routes'][0]['summary']
        print("The Distance between {} and {} is {} KM".format(city,service_city,r["distance"]/1000) )
        time=r["duration"]
        ptime=r["duration"]
        time = float(ptime)
        day = time // (24 * 3600)
        time = time % (24 * 3600)
        hour = time // 3600
        time %= 3600
        minutes = time // 60
        time %= 60
        seconds = time
        s=print("The Duration it will take To reach %d Day %d hour %d Minute %d Seconds"%(day, hour, minutes, seconds))
    
        print(s)
       
    except GeocoderTimedOut as e:
        print(e)
if __name__ == "__main__":
    
    appointment()
    
        
        
        
   



customer Name?rer
customer ID ?67
which  profession service provider you want ?doctor
service provider name ?ripa
Enter starting Date(dd/mm/yyyy):15/12/2019
Hello sir , Our service Providers have their specific Timing, please Choose from there
option-1: 10:00:00 - 12:00:00

option-2: 14:00:00 - 16:00:00

option-3: 18:00:00 - 20:00:00

Enter Your Sloting Option:2
Customer Location(From) :delhi
Service Provider Location(To) :agra
The date you have entered is weekend. Our service provider is not available at that date
